In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Milvus
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import os
from dotenv import load_dotenv

In [10]:
load_dotenv()  # take environment variables from .env.
openai_api_key = os.getenv('OPENAI_API_KEY')
milvus_api_key = os.getenv('MILVUS_API_KEY')

connection_args={
        "uri": "https://in03-5052868020ac71b.api.gcp-us-west1.zillizcloud.com",
        "user": "vaclav@pechtor.ch",
        "token": milvus_api_key,
        "secure": True
    }

In [25]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [26]:
milvus = Milvus(embedding_function=embeddings, collection_name='LuGPT', connection_args=connection_args)

In [31]:
query = 'Wer ist Heinz Bösch?'

In [32]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

from langchain.prompts import PromptTemplate
language = "Swiss German Dialect"
prompt_template = f"""You are an assistant that answers questions about the Kanton Luzern,
 based on given information. Only use the information that was provided below.
 Use the following pieces of context to answer the question at the end.
 If you don't know the answer, just say that you don't know, don't try to make up an answer.

{{context}}

Question: {{question}}
Answer in {language}:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


In [33]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4-0613',temperature=0.0)
qa = RetrievalQA.from_chain_type(llm=llm, 
                                 retriever=milvus.as_retriever(),
                                 chain_type='stuff',
                                 return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs)
result = qa({"query": query})

In [34]:
print(result)

{'query': 'Wer ist Heinz Bösch?', 'result': 'I weiss es nöd, die Information isch nöd vorhande.', 'source_documents': [Document(page_content='Anfrage Bossart Rolf und Mit. über das heutige Verhältnis Berufsbeistände/Privatpersonen\n\nE\n\nMail', metadata={'source': '/var/folders/rp/zgs4ngzd1wg1g91fz8cyj3yw0000gn/T/tmpl48g30tv/www.lu.ch__kr__parlamentsgeschaefte__geschaeftsabo?ges=e24b5d9c1ab74cfe9253cd4f6cd103a2.txt'}), Document(page_content='Barbara Reichlin Radtke\n\nÂ', metadata={'source': '/var/folders/rp/zgs4ngzd1wg1g91fz8cyj3yw0000gn/T/tmpojpb7e32/www.lu.ch__verwaltung__JSD__jsd_unser_departement__jsd_kontakte_karte.txt'}), Document(page_content='Anfrage Koch Hannes und Mit. Ã¼ber die drohende Energiemangellage\n\nE\n\nMail', metadata={'source': '/var/folders/rp/zgs4ngzd1wg1g91fz8cyj3yw0000gn/T/tmpccyptw0q/www.lu.ch__kr__parlamentsgeschaefte__geschaeftsabo?ges=bb26d8cdc5fb4569bfb903705b7393af.txt'}), Document(page_content='Anfrage Bucheli Hanspeter und Mit. Ã¼ber das Bauen in der